# WeChat Tool - Experiment with wxpy Library
### www.KudosData.com
#### By: Sam GU Zhan
#### March, 2017

# Imports

In [ ]:
# coding=UTF-8
from __future__ import division
import re

# Python2 unicode & float-division support:
# from __future__ import unicode_literals, division

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import io

# 中文字符和语言处理库
import jieba

# 机器学习库 sklearn 分类学习模型库
#from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer # 数据结构变换：把 Dict 转换为 稀疏矩阵
# from sklearn.linear_model import LogisticRegression  # 逻辑回归分类模型
# from sklearn.pipeline import make_pipeline # 封装机器学习模型流程
# from sklearn.metrics import confusion_matrix, roc_curve, auc

# 中文显示设置
from pylab import *  
mpl.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体  
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题 
mpl.rcParams['font.size'] = 14 # 设置字体大小

np.random.seed(88)

In [ ]:
import time
from wxpy import *

# Define Functions

In [ ]:
# Python3
# 中文分词功能小函数， 输出 字符串， 各词组由空格分隔
def KudosData_word_tokenizer(foo):
    # remove lead & tail spaces firstly:
    foo = foo.strip()
    seg_token = jieba.cut(str(foo), cut_all=True)
    seg_str = str(' '.join(seg_token)).strip()

    return seg_str
# Python2
# 中文分词功能小函数， 输出 字符串， 各词组由空格分隔
# def KudosData_word_tokenizer(foo):
#     seg_token = jieba.cut(foo, cut_all=True)
#     seg_str = ' '.join(seg_token)
#     return seg_str

In [ ]:
# Python3
# 中文分词功能小函数， 输出 字符串， 各词组由空格分隔
def KudosData_word_count(foo):
    # remove lead & tail spaces firstly:
    foo = foo.strip()
    seg_token = jieba.cut(str(foo), cut_all=True)
    seg_str = str(' '.join(seg_token)).strip()
    seg_count = pd.value_counts(str(seg_str).lower().split(' '))
    seg_count = seg_count.to_dict() 
    seg_count.pop('', None) # remove EMPTY dict key: ''
#     输出 dictionary： { key 词组， value 计数 }
    #     return seg_count.to_dict()
    return seg_count

# Python2
# 中文分词功能小函数， 输出 dictionary： { key 词组， value 计数 }
# def KudosData_word_count(foo):
#     seg_token = jieba.cut(foo, cut_all=True)
#     seg_str = '^'.join(seg_token)
#     seg_count = pd.value_counts(seg_str.lower().split('^'))
#     return seg_count.to_dict()


# Input text

In [ ]:
# process Unicode text input
with io.open('input_text.txt','r',encoding='utf8') as f:
    content = f.read()

title = '''
<Dummy Title>
'''

In [ ]:
# content

In [ ]:
def format_sentence(text):
#     sentence = re.sub(r'\W+', '#', sentence)
    text = re.sub(r'\t+', '', text) # remove one or more Tab
    return text

In [ ]:
def linebreak_conversion_win_linux(text):
    text = re.sub(r'\r', '', text) # remove one or more Windows-line-break
    text = re.sub(r'\u3000', ' ', text) # convert white space: \u3000    
    return text

In [ ]:
def clean_some_whitespace_1(text): # Does not remove normal Space
#     sentence = re.sub(r'\W+', '#', sentence)
    text = re.sub(r'\t+', '', text) # remove one or more Tab
    text = re.sub(r'\f+', '', text) # remove one or more special Space
    text = re.sub(r'\v+', '', text) # remove one or more special Space
    text = re.sub(r' +', ' ', text) # merge two or more Spaces to 1 Space
    
    # remove lead & tail spaces:
    text =text.strip()
    
    return text

In [ ]:
def clean_some_whitespace_2(text): # Does not remove normal Space
#     sentence = re.sub(r'\W+', '#', sentence)
    text = re.sub(r'\n+', ' ', text) # Change one or more \n to a Space, this is to merge sentences within paragraph
    text = re.sub(r' +', ' ', text) # merge two or more Spaces to 1 Space
    text = re.sub(r'(\^\*\#)( +)(\#\*\^)', '^*##*^', text) # remove one or more Spaces between Paragraph-Tags or Sentence-Tags
    
    text = re.sub(r'(\#\*\^S\^\*\#)+', '#*^S^*#', text) # merge two or more sentence-Tags -> 1 Sentence-Tag
    text = re.sub(r'(\#\*\^P\^\*\#)+', '#*^P^*#', text) # merge two or more Paragraph-Tags -> 1 Paragraph-Tag
    
    # remove a Sentence-Tag immediately before a Paragraph-Tag
    text = re.sub(r'(\#\*\^S\^\*\#)( *)(\#\*\^P\^\*\#)', '#*^P^*#', text) 

    # remove lead & tail spaces:
    text =text.strip()
    
    return text

In [ ]:
### Define Paragraph-Tag =  
#   #*^P^*#

### Define Sentence-Tag =  
#   #*^S^*#


In [ ]:
# add a special tag to end of each paragraph
def tag_paragraph(text):
    text = re.sub(r'((\n ) +)+', '#*^P^*#', text) # Tag paragraph, pattern: \n + two or more Spaces
    text = re.sub(r'((\n\t) +)+', '#*^P^*#', text) # Tag paragraph, pattern: \n + two or more Tabs
    text = re.sub(r'(\n( *)\n)+', '#*^P^*#', text) # Tag paragraph, pattern: \n + zero or more Spaces + \n
    text = re.sub(r'(\#\*\^P\^\*\#)+', '#*^P^*#', text) # merge two or more Paragraph-Tags -> 1 Paragraph-Tag

    return text

In [ ]:
# add a special tag to end of each sentence
def tag_sentence(text):
    text = re.sub(r'。+', '。#*^S^*#', text) # Tag sentence - Chinese
    text = re.sub(r'！+', '！#*^S^*#', text) # Tag sentence - Chinese
    text = re.sub(r'\？+', '？#*^S^*#', text) # Tag sentence - Chinese
#     text = re.sub(r'；+', '；#*^S^*#', text) # Tag sentence - Chinese

    # 2017 MAR 24
    text = re.sub(r'(\.)( +)', '.#*^S^*#', text) # Tag sentence - English
    text = re.sub(r'(!)( +)', '!#*^S^*#', text) # Tag sentence - English
    text = re.sub(r'\?( +)', '?#*^S^*#', text) # Tag sentence - English
#     text = re.sub(r'(;)( +)', ';#*^S^*#', text) # Tag sentence - English

    text = re.sub(r'\.\n', '.#*^S^*#', text) # Tag sentence - English
    text = re.sub(r'!\n', '!#*^S^*#', text) # Tag sentence - English
    text = re.sub(r'\?\n', '?#*^S^*#', text) # Tag sentence - English
#     text = re.sub(r';\n', ';#*^S^*#', text) # Tag sentence - English
    
    # remove a Sentence-Tag immediately before an ending quotation
    text = re.sub(r'\#\*\^S\^\*\#’', '’#*^S^*#', text) # Chinese ’
    text = re.sub(r'\#\*\^S\^\*\#”', '”#*^S^*#', text) # Chinese ”
    text = re.sub(r'\#\*\^S\^\*\#\'', '\'#*^S^*#', text) # English '
    text = re.sub(r'\#\*\^S\^\*\#"', '"#*^S^*#', text) # English "
    
    return text

### Start tagging:

In [ ]:
content_format = linebreak_conversion_win_linux(content)
# content_format

In [ ]:
content_format = tag_paragraph(content_format)
# content_format

In [ ]:
print(len(content_format))

In [ ]:
content_format = clean_some_whitespace_1(content_format)
# content_format

In [ ]:
print(len(content_format))

In [ ]:
content_format = tag_sentence(content_format)
# content_format

In [ ]:
print(len(content_format))

In [ ]:
content_format = clean_some_whitespace_2(content_format)
# content_format

In [ ]:
print(len(content_format))


### Transfer tagged text to Pandas Dataframe

In [ ]:
# Split a text into paragraphs
def split_article_to_paragraphs(text):
#     text = text.replace("#*^P^*#", "#*^S^*#") # convert Paragraph-Tag        
    return text.split("#*^P^*#")


In [ ]:
# Split a paragraph into sentences
def split_paragraph_to_sentences(text):
#     text = text.replace("#*^P^*#", "#*^S^*#") # convert Paragraph-Tag        
    return text.split("#*^S^*#")


In [ ]:
# 1st loop Paragraphs list, 2nd loop Sentences list
# create a few new columns, then write into dataframe, together with original Sentence string

# define empty dataframe:
df_article = pd.DataFrame(columns=('sentence', 
                                   'word_count', # sentence word count, including punctuations 
                                   'sentence_id', # unique sentence s/n within an article
                                   'sentence_id_paragraph',  # sentence s/n within a paragraph 
                                   'paragraph_id', 
                                   'class_rank', 
                                   'score_word', # score based on word tf-idf
                                   'score_sentence', # score based on intersection of sentence pairs
                                   'score_word_norm', # Normalized score
                                   'score_sentence_norm', # Normalized score
                                   'score',
                                  ))
df_sentence_id = 0

# split_article_to_paragraphs:
article_paragraphs = split_article_to_paragraphs(content_format)

for i in range(0, len(article_paragraphs)):
    # split_paragraph_to_sentences:
    article_paragraphs_sentences = split_paragraph_to_sentences(article_paragraphs[i].strip())

    for j in range(0, len(article_paragraphs_sentences)):
        if article_paragraphs_sentences[j].strip() != '':
            df_sentence_id = df_sentence_id + 1
            # write to dataframe:
            df_article.loc[len(df_article)] = [article_paragraphs_sentences[j].strip(), 
                                               len(article_paragraphs_sentences[j].strip()), 
                                               df_sentence_id, 
                                               j+1, 
                                               i+1, 
                                               '', 
                                               '', 
                                               '', 
                                               '', 
                                               '', 
                                               '']


In [ ]:
# assume the 1st sentence as Title of Article

title = df_article['sentence'][0]
print('Title of Article : ', title)

In [ ]:
df_article['sentence'].head()

In [1]:
from wxpy import *

In [2]:
# 初始化机器人，扫码登陆
bot = Bot()

Getting uuid of QR code.
INFO:itchat:Getting uuid of QR code.
INFO:itchat:Downloading QR code.
Please scan the QR code to log in.
INFO:itchat:Please scan the QR code to log in.
Please press confirm on your phone.
INFO:itchat:Please press confirm on your phone.
Loading the contact, this may take a little while.
INFO:itchat:Loading the contact, this may take a little while.
Login successfully as 白黑
INFO:itchat:Login successfully as 白黑


In [91]:
# 搜索好友名称
# my_friend = bot.friends().search('hss7777777')[0]
my_friend = bot.friends().search('郝素素')[0]
# my_friend = bot.friends().search('王嘉祎')[0]
# my_friend = bot.friends().search('白黑')[0]

my_friend

<Friend: 郝素素>

In [92]:
@bot.register()
def just_print(msg):
    # 打印消息
    print(msg)

In [93]:
@bot.register([my_friend, Group])
# @bot.register([my_friend, Group], None, False)
# @bot.register()
# @bot.register([Group], TEXT, False)

def auto_reply(msg):
    # 如果是群聊，但没有被 @，则不回复
    if isinstance(msg.chat, Group) and not msg.is_at:
        return
    else:
        # 回复消息内容和类型
        return '<自动回复>\n已收到消息:\n\"{}\"\n消息类型：\n({})'.format(msg.text, msg.type)

In [96]:
bot.registered.enabled

[<MessageConfig: 白黑: just_print (Enabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Enabled, Async)>]

ERROR:wxpy.api.bot:
An error occurred in <function auto_reply at 0x00000268B5308510>.
Traceback (most recent call last):
  File "G:\Tool_PGM\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "G:\Tool_PGM\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py", line 345, in _make_request
    self._validate_conn(conn)
  File "G:\Tool_PGM\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py", line 844, in _validate_conn
    conn.connect()
  File "G:\Tool_PGM\Anaconda3\lib\site-packages\requests\packages\urllib3\connection.py", line 326, in connect
    ssl_context=context)
  File "G:\Tool_PGM\Anaconda3\lib\site-packages\requests\packages\urllib3\util\ssl_.py", line 324, in ssl_wrap_socket
    return context.wrap_socket(sock, server_hostname=server_hostname)
  File "G:\Tool_PGM\Anaconda3\lib\ssl.py", line 376, in wrap_socket
    _context=self)
  File "G:\Tool_PGM\Anaconda3\lib\ssl.py

In [89]:
bot.registered.disable()

In [90]:
bot.registered.enabled

[]

In [95]:
bot.registered

[<MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: just_print (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: just_print (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageConfig: 白黑: auto_reply (Disabled, Async)>,
 <MessageCon

In [97]:
# 堵塞线程，并进入 Python 命令行
# embed()

In [ ]:
# my_friend.send('<程序自动发送>\n<第%d条，共%d条>\n%s' % (1, len(df_article['sentence']), df_article['sentence'][100]))
# my_friend.send('<程序自动发送>\n恭喜恭喜！\n红包拿来~')

In [ ]:
# 发送文本给好友
count = len(df_article['sentence'])
# count = 5
for i in range(729, count):
    try:
        my_friend.send('<第%d条，共%d条>\n%s' % (i+1, count, df_article['sentence'][i]))
        print ('success rec_id = %d' % (i+1))
        time.sleep(np.random.randint(low = 5, high = 20))
    except:
        tmp_rand_int = np.random.randint(low = 70, high = 130)
        print ('retry   rec_id = %d, after %d seconds... ' % (i+1, tmp_rand_int))
        time.sleep(tmp_rand_int)
        try:
            my_friend.send('<第%d条，共%d条>\n%s' % (i+1, count, df_article['sentence'][i]))
            print ('success rec_id = %d' % (i+1))
            time.sleep(np.random.randint(low = 5, high = 20))
        except:
            tmp_rand_int = np.random.randint(low = 70, high = 130)
            print ('retry   rec_id = %d, after %d seconds... ' % (i+1, tmp_rand_int))
            time.sleep(tmp_rand_int)
            my_friend.send('<第%d条，共%d条>\n%s' % (i+1, count, df_article['sentence'][i]))
            print ('success rec_id = %d' % (i+1))
            time.sleep(np.random.randint(low = 5, high = 20))


In [ ]:
    try:
        my_friend.send('<第%d条，共%d条>\n%s' % (i+1, count, df_article['sentence'][i]))
        print ('success rec_id = %d' % (i+1))
        time.sleep(np.random.randint(low = 5, high = 20))
    except:
        tmp_rand_int = np.random.randint(low = 70, high = 130)
        print ('retry   rec_id = %d, after %d seconds... ' % (i+1, tmp_rand_int))
        time.sleep(tmp_rand_int)
        my_friend.send('<第%d条，共%d条>\n%s' % (i+1, count, df_article['sentence'][i]))
        print ('success rec_id = %d' % (i+1))
        time.sleep(np.random.randint(low = 5, high = 20))

In [ ]:
# 发送文本给好友
count = len(df_article['sentence'])
# count = 10
for i in range(0, count):
    my_friend.send('<第%d条，共%d条>\n%s' % (i+1, count, df_article['sentence'][i]))
    print (i+1)
#     my_friend.send('<程序自动发送> \n恭喜恭喜！\n红包拿来~')
    time.sleep(np.random.randint(low = 10, high = 30))

### word_tokenizer

In [ ]:
# KudosData_word_tokenizer
df_article['sentence_tokenized'] = df_article['sentence'].apply(lambda x: KudosData_word_tokenizer(x))

In [ ]:
# Make sure no empty sentences:
print('Number of empty sentences in dataframe: %d ' % len(df_article[df_article['sentence_tokenized'] == '']))

In [ ]:
# Remove invalid empty sentences
print(len(df_article))
df_article=df_article[df_article['sentence_tokenized'] != '']
df_article = df_article.sort_values(by=['sentence_id'],).reset_index(drop=True)
print(len(df_article))

### Term Frequency

In [ ]:
# KudosData_word_count
df_article['sentence_tf'] = df_article['sentence'].apply(lambda x: KudosData_word_count(x))

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df_article['sentence_tokenized']

vectorizer = TfidfVectorizer()

# my_stopword_list = ['and','to','the','of', 'in']
#vectorizer = TfidfVectorizer(stop_words=my_stopword_list)

# choice of no nomalization of tfidf output (not recommended)
#vectorizer = TfidfVectorizer(norm=None)

# TF-IDF score
tfidf = vectorizer.fit_transform(corpus)

# IDF score
idf_dict = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

# TF is in df_article[['sentence_tf']]

In [ ]:
### 把TF-iDF数值赋予相对应的词组
tfidf = tfidf.tocsr()

n_docs = tfidf.shape[0]
tfidftables = [{} for _ in range(n_docs)]
terms = vectorizer.get_feature_names()

for i, j in zip(*tfidf.nonzero()):
    tfidftables[i][terms[j]] = tfidf[i, j]

In [ ]:
# Document-Term-Matrix's TF-IDF matrix size:
print ('This tfidf matrix is a very large table: [ %d rows/docs X %d columns/words ]' 
       % (tfidf.shape[0], tfidf.shape[1]))
print ('It contains %d eliments: one score per word per document !'
       % (tfidf.shape[0] * tfidf.shape[1]))

In [ ]:
# Add tfidf score into dataframe 
df_article['tfidf'] = tfidftables

In [ ]:
# df_article[['sentence', 'sentence_tokenized', 'sentence_tf', 'tfidf']]

# Calculate importance score for each sentence

### Scoring (1)
### Calculate score_word for each sentence, based on sentence word_count tf-idf:

In [ ]:
# experiment: use tf-idf and len(sentence_tokenized) to calculate score
# tmp_mean = tmp_sum / len(df_article['sentence_tokenized'][i])

for i in range(0,len(df_article)):
    if len(df_article['tfidf'][i]) == 0:
        df_article['score_word'][i] = 0
    else:
        tmp_sum = 0
        for key, values in df_article['tfidf'][i].items():
            tmp_sum += values
        
        tmp_mean = tmp_sum / len(df_article['sentence_tokenized'][i])
        df_article['score_word'][i] = tmp_mean 


### Scoring (2)
### Calculate score_sentence for each sentence, based on pair-wise sentence comparison/intersection:

In [ ]:
# Caculate raw intersection score between pair of two sentences, from df_article['sentence_tokenized']
def sentences_intersection(sent1tokenized, sent2tokenized):
    # www.KudosData.com - Chinese
    # split the sentence into words/tokens
    s1 = set(sent1tokenized.split(" "))
    s2 = set(sent2tokenized.split(" "))

    # If there is not intersection, just return 0
    if (len(s1) + len(s2)) == 0:
        print('# If there is not intersection, just return 0')
        return 0

    # Normalize the result by the average number of words
    return len(s1.intersection(s2)) / ((len(s1) + len(s2)) / 2)


### Below step runs long time... Tuning needed

In [ ]:
# Calculate important score of every pair of sentences

n = len(df_article['sentence_tokenized'])
        
# [Sam python 2.7 -> 3.4] values = [[0 for x in xrange(n)] for x in xrange(n)]
df_score_raw_values = [[0 for x in range(n)] for x in range(n)]
for i in range(0, n):
    for j in range(0, n):
        df_score_raw_values[i][j] = sentences_intersection(df_article['sentence_tokenized'][i], 
                                                           df_article['sentence_tokenized'][j])

# The score of a sentence is the sum of all its intersection
sentences_dic = {}

for i in range(0, n):
    df_score = 0
    for j in range(0, n):
        if i == j:
            continue
        df_score += df_score_raw_values[i][j]
    df_article['score_sentence'][i] = df_score


### Visualize Data (Internal use,  not for production)

In [ ]:
df_article

In [ ]:
# 图表显示：
plt.figure(figsize=(16, 4))
plt.title(u'图')  
plt.xlabel(u'X坐标：Sentence word_count')  
plt.ylabel(u'Y坐标：Sentence frequency')  
# df_article['word_count'].value_counts().sort_values(ascending=False).plot(kind='bar', color='green')
df_article['word_count'].hist(bins=100)
# plt.gca().invert_yaxis()
plt.show()


In [ ]:
# 图表显示：
plt.figure(figsize=(16, 4))
plt.title(u'图')  
plt.xlabel(u'X坐标：Paragraph_id')  
plt.ylabel(u'Y坐标：Sentence frequency')  
df_article['paragraph_id'].hist(bins=100)
# plt.gca().invert_yaxis()
plt.show()


In [ ]:
# 图表显示：
plt.figure(figsize=(16, 4))
plt.title(u'图')  
plt.xlabel(u'X坐标：sentence_id_paragraph')  
plt.ylabel(u'Y坐标：Sentence frequency')  
df_article['sentence_id_paragraph'].hist(bins=100)
# plt.gca().invert_yaxis()
plt.show()


In [ ]:
# 图表显示：
plt.figure(figsize=(16, 4))
plt.title(u'图')  
plt.xlabel(u'X坐标：score_word')  
plt.ylabel(u'Y坐标：frequency')  
df_article['score_word'].hist(bins = 100)
# plt.xscale('log')
# plt.yscale('log')
#plt.xlim(0,0.5)
#plt.ylim(0,0.5)
# plt.gca().invert_yaxis()
plt.show()


In [ ]:
# 图表显示：
plt.figure(figsize=(16, 4))
plt.title(u'图')  
plt.xlabel(u'X坐标：score_sentence')  
plt.ylabel(u'Y坐标：frequency')  
df_article['score_sentence'].hist(bins = 100)
# plt.xscale('log')
# plt.yscale('log')
#plt.xlim(0,0.5)
#plt.ylim(0,0.5)
# plt.gca().invert_yaxis()
plt.show()


In [ ]:
# df_article[(df_article['score_word'] > 0.15) & (df_article['score_word'] < 0.25)]
# df_article[(df_article['score_word'] > 0.2)].sort_values(by=['score_sentence', 'score_word'], ascending=[False, False,])
# df_article[(df_article['score_sentence'] > 250)].sort_values(by=['score_word', 'score_sentence'], ascending=[False, False,])


### Score Normalization

In [ ]:
# log(score_word)
df_article['score_word_log'] = np.log(df_article['score_word'].astype('float64') + 
                                      df_article[df_article['score_word'] >0 ]['score_word'].min()/2)


In [ ]:
# Normalize score_word_log - Zero mean, unit variance

# df_article['score_word_norm'] = (df_article['score_word'] - df_article['score_word'].mean()) / df_article['score_word'].std()
df_article['score_word_norm'] = (df_article['score_word_log'] - df_article['score_word_log'].mean()) / df_article['score_word_log'].std()

In [ ]:
df_article['score_word_norm'].hist(bins=100)

In [ ]:
# Normalize score_sentence - Zero mean, unit variance

df_article['score_sentence_norm'] = (df_article['score_sentence'] - df_article['score_sentence'].mean()) / df_article['score_sentence'].std()

In [ ]:
df_article['score_sentence_norm'].hist(bins=100)

### Generate class_rank

In [ ]:
# Score integration
# df_article['score'] = (df_article['score_sentence_norm'] + df_article['score_word_norm']) / 2

# Sam Gu: 23 Mar 2017 - Experiment found that the score_word, which is based on tf-idf, doesn't seem to work well.
#                       score_word     tends to favor short sentences
#                       score_sentence tends to favor long  sentences
#                       Hence, here we use score_sentence only for final scoring.

# df_article['score'] = (df_article['score_word'] + df_article['score_sentence'] ) / 2
df_article['score'] = df_article['score_sentence']

In [ ]:
# Min-Max normalization:
df_article['score'] = (df_article['score'] - df_article['score'].min()) / (df_article['score'].max() -df_article['score'].min())

In [ ]:
df_article['score'].hist(bins=100)

In [ ]:
# sort firstly
df_article = df_article.sort_values(by=['paragraph_id', 'score'], ascending=[True, False]).reset_index(drop=True)

### Below step runs long time... Tuning needed

In [ ]:
# Generate Class_Rank

current_class_rank = 0
current_paragraph_id = 0

for i in range(0, len(df_article)):
    if df_article['paragraph_id'][i] != current_paragraph_id: # change of Paragraph, thus reset class_rank
        current_class_rank = 1
        current_paragraph_id = df_article['paragraph_id'][i]
    else:
        current_class_rank = current_class_rank + 1
        
    df_article['class_rank'][i] = current_class_rank


In [ ]:
# sort Dataframe to 'result lookup mode'
df_article = df_article.sort_values(by=['class_rank', 'score', 'paragraph_id', 'sentence_id'], 
                                    ascending=[True, False, True, True]).reset_index(drop=True)

In [ ]:
df_article[['sentence',
           'paragraph_id',
           'sentence_id_paragraph',
           'class_rank',
           'score',
           'sentence_tokenized'
          ]]

In [ ]:
# df_article[(df_article['score'] == 0) | (df_article['score'] == 1)]

# Extract results based on user parameters:
* Max number of words
* % of original number of words
* Max lines of sentences


In [ ]:
# Make a dataframe copy
# Currently, the two dataframes are exactly the same.
df_article_internal = pd.DataFrame.copy(df_article)

In [ ]:
total_words_original_article = df_article['sentence'].map(len).sum()
total_words_internal_article = df_article_internal['sentence'].map(len).sum()
# total_words_article_summary  = df_article_final['sentence'].map(len).sum()

# print('total_words_original_article : ', total_words_original_article)
# print('total_words_internal_article : ', total_words_internal_article)
# print('total_words_article_summary  : ', total_words_article_summary)


In [ ]:
# Sam Gu: experiment shows no major improvement to use code in this block:

'''

# Heuristic cleaning:
# 1.Remove sentences, which has only one valid word. 
# 2.Remove paragraph, which has only single sentence.

# 1.
df_article_internal = df_article_internal[df_article_internal['sentence_tokenized'].map(len) > 1]
print('*** www.KudosData.com *** Removed number of sentences, which has only one valid word : %d'
      % (len(df_article) - len(df_article_internal)))

# 2.
df_article_internal_paragraph = df_article_internal['paragraph_id'].value_counts().to_frame(name = 'sentence_count')
df_article_internal_paragraph = df_article_internal_paragraph[df_article_internal_paragraph['sentence_count'] > 1]
valid_paragraph_id = df_article_internal_paragraph.index.tolist()
df_article_internal = df_article_internal[df_article_internal['paragraph_id'].isin(valid_paragraph_id)] 
print('*** www.KudosData.com *** Removed number of sentences in total : %d' % (len(df_article) - len(df_article_internal)))

# sort Dataframe to 'result lookup mode'
df_article_internal = df_article_internal.sort_values(by=['class_rank', 'score', 'paragraph_id', 'sentence_id'], 
                                    ascending=[True, False, True, True]).reset_index(drop=True)
# Above sort a must sort !!! for below processing:
# Loop Dataframe, accumulate length of sentences, stop when parm_max_word reached, return the index(), cut dataframe to display

'''

### Accept user parameters:

In [ ]:
# valid range: >= 0
parm_max_word = 200

# valid range: >= 0
parm_max_sentence = 10

# valid range: [0, 100%]
parm_max_percent = 0.01

### Validation of user parameters:

In [ ]:
if (isinstance(parm_max_word, int) | isinstance(parm_max_word, float)):
    if parm_max_word >= 0:
        print('!1! valid input parm_max_word : ', parm_max_word)
    else:
        print('!2! Invalid input parm_max_word : ', parm_max_word)    
else:
    print('!3! Invalid input parm_max_word : ', parm_max_word)

if (isinstance(parm_max_sentence, int) | isinstance(parm_max_sentence, float)):
    if parm_max_sentence >= 0:
        print('!1! valid input parm_max_sentence : ', parm_max_sentence)
    else:
        print('!2! Invalid input parm_max_sentence : ', parm_max_sentence)    
else:
    print('!3! Invalid input parm_max_sentence : ', parm_max_sentence)

if (isinstance(parm_max_percent, int) | isinstance(parm_max_percent, float)):
    if parm_max_percent >= 0:
        print('!1! valid input parm_max_percent : ', parm_max_percent)
    else:
        print('!2! Invalid input parm_max_percent : ', parm_max_percent)    
else:
    print('!3! Invalid input parm_max_percent : ', parm_max_percent)


In [ ]:
# cut by parm_max_percent

# Loop Dataframe, accumulate length of sentences, stop when parm_max_word reached, return the index(), cut dataframe to display

sum_current_word = 0
cut_index = len(df_article_internal['sentence'])

# print('Start loop...')
for s in range(0, len(df_article_internal['sentence'])):
#     print('s : %d' % s)
    if sum_current_word / total_words_original_article <= parm_max_percent:
        sum_current_word += len(df_article_internal['sentence'][s])
    else:
#         stop, return index number
        cut_index = s - 1
        sum_current_word -= len(df_article_internal['sentence'][s-1])

#         print('To break')
        break

# print('End loop')
sum_current_percent = sum_current_word / total_words_original_article
print('---------- cut by parm_max_percent :')
print('sum_current_word  / total_words_original_article:', sum_current_percent)
print('cut_index : ', cut_index)

In [ ]:
# cut by parm_max_word

# Loop Dataframe, accumulate length of sentences, stop when parm_max_word reached, return the index(), cut dataframe to display

sum_current_word = 0
cut_index = len(df_article_internal['sentence'])

# print('Start loop...')
for s in range(0, len(df_article_internal['sentence'])):
#     print('s : %d' % s)
    if sum_current_word <= parm_max_word:
        sum_current_word += len(df_article_internal['sentence'][s])
    else:
#         stop, return index number
        cut_index = s - 1
        sum_current_word -= len(df_article_internal['sentence'][s-1])
#         print('To break')
        break

# print('End loop')
print('---------- cut by parm_max_word :')
print('sum_current_word :', sum_current_word)
print('cut_index : ', cut_index)

In [ ]:
# cut by parm_max_sentence

cut_index = parm_max_sentence

print('---------- cut by parm_max_sentence :')
print('cut_index : ', cut_index)

In [ ]:
# Extract top number of sentences as summary, based on: cut_index
df_article_final = df_article_internal[0:cut_index]

In [ ]:
# sort by original sentence order 
df_article_final = df_article_final.sort_values(by=['sentence_id'], ascending=[True])
df_article_final[['sentence_id', 'sentence', 'score', 'class_rank', 'paragraph_id', 'sentence_id_paragraph']]

In [ ]:
# total_words_original_article = df_article['sentence'].map(len).sum()
# total_words_internal_article = df_article_internal['sentence'].map(len).sum()
total_words_article_summary  = df_article_final['sentence'].map(len).sum()

print('total_words_original_article : ', total_words_original_article)
print('total_words_internal_article : ', total_words_internal_article)
print('total_words_article_summary  : ', total_words_article_summary)


# Output results to a file

In [ ]:
# print('\n'.join(list(df_article_final['sentence'])))

In [ ]:

with io.open('output_topic_summary.txt','w',encoding='utf8') as f:
#     f.write("Original Length : %s" % total_words_original_article)
    f.write("No. Paragraphs  : %d" % df_article_internal['paragraph_id'].max())
    f.write("\n")
    f.write("Original Length : %s" % total_words_internal_article)
    f.write("\n")
    f.write("Summary  Length : %s" % total_words_article_summary)
    f.write("\n")
#     f.write("Summary  Ratio  : %s %%" % (100 * (sum_current_word / total_words_original_article)))
    f.write("Summary  Ratio  : %.2f %%" % (100 * (total_words_article_summary / total_words_internal_article)))
    f.write("\n")
    f.write("\n")
    f.write("Title of Article: %s" % title)
    f.write("\n")
    f.write("\n")
    f.write('\n'.join(list(df_article_final['sentence'])))
    f.close()
    

In [ ]:
# This is to check if there is sentence with very few valid/real word, should have very low score.
df_article[['sentence', 'word_count', 'sentence_tokenized', 'tfidf', 'score']][df_article['sentence_tokenized'].map(len) <= 3]